In [14]:
from langchain_community.document_loaders import PyPDFLoader
import json

# Загружаем пдф. это книжка про растения
loader = PyPDFLoader(
    file_path="docs_3_2/Теневые сады.pdf",
    # mode="page",  # "page" - по страницам (по умолчанию), "single" - весь документ
    extract_images=False,  # извлекать изображения
)

docs = loader.load()

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# разбиваем на куски
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)

splitted_docs = text_splitter.split_documents(docs)

print(f"Было документов: {len(docs)}, стало фрагментов: {len(splitted_docs)}")

Было документов: 143, стало фрагментов: 728


In [32]:
from langchain_huggingface import HuggingFaceEmbeddings
# загрузить модель эмбеддингов
embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [40]:
from langchain_community.vectorstores import FAISS
# заполняем базу данных эмбеддингами кусков
vector_store = FAISS.from_documents(splitted_docs, embed_model)

# сохранение
vector_store.save_local("index/my_faiss_index")

# загрузка
new_store = FAISS.load_local("index/my_faiss_index", embed_model, allow_dangerous_deserialization=True)

In [41]:
# с помощью similarity_search ищем самый близкий чанк. и выводим его и его метаданные
def return_chunk(query):
    docs_found =  new_store.similarity_search(query, k=1)
    metadata = docs_found[0].metadata
    txt = docs_found[0].page_content
    print(json.dumps(metadata, indent=4, ensure_ascii=False))
    print(txt)

In [42]:
# далее три примера работы. в целом конечно плохо. часть не попадает , даже если я вставляю копию текста из книги. показываю где все таки совпало.
return_chunk("Красоднев малый – Hemerocallis minor Miller")

{
    "producer": "Microsoft® Word 2010",
    "creator": "Microsoft® Word 2010",
    "creationdate": "2019-04-24T14:46:21+07:00",
    "subject": "Публикации ТГУ",
    "author": "Прокопьев А.С.",
    "keywords": "Учебно-методическое пособие содержит информацию о растительных композициях, которые можно создавать на участках различной степени зате-\r\nнения. Приведены основные принципы и распространенные трудности обу-\r\nстройства теневых садов. Для озеленения в условия",
    "moddate": "2019-08-05T13:34:37+07:00",
    "title": "Теневые сады: учебно-методическое пособие",
    "source": "docs_3_2/Теневые сады.pdf",
    "total_pages": 143,
    "page": 100,
    "page_label": "101"
}
101 
Красоднев малый – Hemerocallis minor Miller 
Семейство Лилейные (Liliaceae) 
 
Другие названия: лилейник малый. 
Распространение: Сибирь, Д. Восток, Монголия, Китай, Корея.


In [43]:
return_chunk("Горечавка семираздельная – Gentiana septemfida Pall.")

{
    "producer": "Microsoft® Word 2010",
    "creator": "Microsoft® Word 2010",
    "creationdate": "2019-04-24T14:46:21+07:00",
    "subject": "Публикации ТГУ",
    "author": "Прокопьев А.С.",
    "keywords": "Учебно-методическое пособие содержит информацию о растительных композициях, которые можно создавать на участках различной степени зате-\r\nнения. Приведены основные принципы и распространенные трудности обу-\r\nстройства теневых садов. Для озеленения в условия",
    "moddate": "2019-08-05T13:34:37+07:00",
    "title": "Теневые сады: учебно-методическое пособие",
    "source": "docs_3_2/Теневые сады.pdf",
    "total_pages": 143,
    "page": 66,
    "page_label": "67"
}
67 
Горечавка семираздельная – Gentiana septemfida Pall. 
Семейство Горечавковые (Gentianaceae) 
 
Другие названия: сокольница семираздельная. 
Распространение: Кавказ, Сибирь, Ср. Азия.


In [44]:
return_chunk("Малые архитектурные формы (МАФ)")

{
    "producer": "Microsoft® Word 2010",
    "creator": "Microsoft® Word 2010",
    "creationdate": "2019-04-24T14:46:21+07:00",
    "subject": "Публикации ТГУ",
    "author": "Прокопьев А.С.",
    "keywords": "Учебно-методическое пособие содержит информацию о растительных композициях, которые можно создавать на участках различной степени зате-\r\nнения. Приведены основные принципы и распространенные трудности обу-\r\nстройства теневых садов. Для озеленения в условия",
    "moddate": "2019-08-05T13:34:37+07:00",
    "title": "Теневые сады: учебно-методическое пособие",
    "source": "docs_3_2/Теневые сады.pdf",
    "total_pages": 143,
    "page": 126,
    "page_label": "127"
}
ществ. 
Малые архитектурные формы (МАФ)  – искусственные 
элементы садово-парковой композиции (скамьи, фонари, бесе д-
ки и др.), устанавливаемые в садах и парк ах как в функциональ-
